In [2]:
#import libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import glob
import os

In [3]:
data_path = "/home/lenka/code/kai-pre/sales-ninja/raw_data"
csv_files = glob.glob(os.path.join(data_path, "*.csv"))
datasets = {}
for file in csv_files:
    name = os.path.splitext(os.path.basename(file))[0]  
    datasets[name] = pd.read_csv(file)

In [4]:
fact_sales = datasets["FactSales"]
dim_date = datasets["DimDate"]
dim_product = datasets["DimProduct"]
fact_online_sales = datasets["FactOnlineSales"]
dim_geo = datasets["DimGeography"]
dim_promotion = datasets["DimPromotion"]
dim_sub_cat = datasets["DimProductSubcategory"]
dim_product_cat = datasets["DimProductCategory"]
dim_channel =  datasets["DimChannel"]
dim_store =  datasets["DimStore"]

In [6]:
fact_sales.head()

,SalesKey,DateKey,channelKey,StoreKey,ProductKey,PromotionKey,CurrencyKey,UnitCost,UnitPrice,SalesQuantity,ReturnQuantity,ReturnAmount,DiscountQuantity,DiscountAmount,TotalCost,SalesAmount
0,1,2007-01-02,1,209,956,10,1,91.05,198.0,8,0,0.0,1,39.600,728.40,1544.400
1,2,2007-02-12,4,308,766,2,1,10.15,19.9,4,0,0.0,1,0.995,40.60,78.605
2,3,2008-01-24,1,156,1175,11,1,209.03,410.0,9,0,0.0,3,61.500,1881.27,3628.500
3,4,2008-01-13,2,306,1429,10,1,132.90,289.0,8,0,0.0,1,57.800,1063.20,2254.200
4,5,2008-01-22,2,306,1133,10,1,144.52,436.2,24,0,0.0,3,261.720,3468.48,10207.080


In [7]:
fact_sales = datasets["FactSales"]
fact_sales["DateKey"] = pd.to_datetime(fact_sales["DateKey"])
datasets["DimDate"]["DateKey"] = pd.to_datetime(datasets["DimDate"]["DateKey"])
fact_sales.rename(columns={"channelKey": "ChannelKey"}, inplace=True)

In [8]:
merged_data = fact_sales.merge(datasets["DimDate"], on="DateKey", how="left")
merged_data = merged_data.merge(datasets["DimProduct"], on="ProductKey", how="left")
merged_data = merged_data.merge(datasets["DimStore"], on="StoreKey", how="left")
merged_data = merged_data.merge(datasets["DimProductSubcategory"], on="ProductSubcategoryKey", how="left")
merged_data = merged_data.merge(datasets["DimProductCategory"], on="ProductCategoryKey", how="left")
merged_data = merged_data.merge(datasets["DimPromotion"], on="PromotionKey", how="left")
merged_data = merged_data.merge(datasets["DimChannel"], on="ChannelKey", how="left")
merged_data = merged_data.merge(datasets["DimGeography"], on="GeographyKey", how="left")

In [10]:
merged_data.columns

Index(['SalesKey', 'DateKey', 'ChannelKey', 'StoreKey', 'ProductKey',
       'PromotionKey', 'CurrencyKey', 'UnitCost_x', 'UnitPrice_x',
       'SalesQuantity', 'ReturnQuantity', 'ReturnAmount', 'DiscountQuantity',
       'DiscountAmount', 'TotalCost', 'SalesAmount', 'CalendarYear',
       'CalendarYearLabel', 'CalendarHalfYearLabel', 'CalendarQuarterLabel',
       'CalendarMonthLabel', 'CalendarWeekLabel', 'CalendarDayOfWeekLabel',
       'FiscalYear', 'FiscalYearLabel', 'FiscalHalfYearLabel',
       'FiscalQuarterLabel', 'FiscalMonthLabel', 'IsWorkDay', 'IsHoliday',
       'EuropeSeason', 'NorthAmericaSeason', 'AsiaSeason', 'MonthNumber',
       'CalendarDayOfWeekNumber', 'ProductName', 'ProductDescription',
       'ProductSubcategoryKey', 'Manufacturer', 'BrandName', 'ClassID',
       'ClassName', 'StyleID', 'StyleName', 'ColorID', 'ColorName', 'Weight',
       'WeightUnitMeasureID', 'UnitOfMeasureID', 'UnitOfMeasureName',
       'StockTypeID', 'StockTypeName', 'UnitCost_y', 'UnitPr

In [11]:
#get rid of duplicates
merged_data = merged_data.loc[:, ~merged_data.columns.str.endswith('_y')]
merged_data.columns = [col.replace('_x', '') if col.endswith('_x') else col for col in merged_data.columns]

In [12]:
merged_data.columns


Index(['SalesKey', 'DateKey', 'ChannelKey', 'StoreKey', 'ProductKey',
       'PromotionKey', 'CurrencyKey', 'UnitCost', 'UnitPrice', 'SalesQuantity',
       'ReturnQuantity', 'ReturnAmount', 'DiscountQuantity', 'DiscountAmount',
       'TotalCost', 'SalesAmount', 'CalendarYear', 'CalendarYearLabel',
       'CalendarHalfYearLabel', 'CalendarQuarterLabel', 'CalendarMonthLabel',
       'CalendarWeekLabel', 'CalendarDayOfWeekLabel', 'FiscalYear',
       'FiscalYearLabel', 'FiscalHalfYearLabel', 'FiscalQuarterLabel',
       'FiscalMonthLabel', 'IsWorkDay', 'IsHoliday', 'EuropeSeason',
       'NorthAmericaSeason', 'AsiaSeason', 'MonthNumber',
       'CalendarDayOfWeekNumber', 'ProductName', 'ProductDescription',
       'ProductSubcategoryKey', 'Manufacturer', 'BrandName', 'ClassID',
       'ClassName', 'StyleID', 'StyleName', 'ColorID', 'ColorName', 'Weight',
       'WeightUnitMeasureID', 'UnitOfMeasureID', 'UnitOfMeasureName',
       'StockTypeID', 'StockTypeName', 'AvailableForSaleDate', '

In [13]:
#KPI Revenue
quarterly_revenue = merged_data.groupby("CalendarQuarterLabel")["SalesAmount"].sum().reset_index()
quarterly_revenue.columns = ["Quarter", "TotalRevenue"]

In [15]:
#KPI Revenue
monthly_revenue = merged_data.groupby(["MonthNumber", "CalendarMonthLabel"])["SalesAmount"].sum().reset_index()
monthly_revenue = monthly_revenue.sort_values("MonthNumber")
monthly_revenue.columns = ["MonthNumber", "Month", "TotalRevenue"]

In [16]:
#KPI Revenue
daily_revenue = merged_data.groupby("DateKey")["SalesAmount"].sum().reset_index()
daily_revenue.columns = ["Date", "TotalRevenue"]

In [18]:
#KPI Net Sales
merged_data["NetSales"] = (
    merged_data["SalesAmount"]
    - merged_data["ReturnAmount"]
    - merged_data["DiscountAmount"]
)
#daily 
daily_net_sales = (
    merged_data.groupby("DateKey")["NetSales"]
    .sum()
    .reset_index()
    .rename(columns={"DateKey": "Date"})
)
#monthly
monthly_net_sales = (
    merged_data.groupby(["CalendarYear", "MonthNumber", "CalendarMonthLabel"])["NetSales"]
    .sum()
    .reset_index()
    .sort_values(["CalendarYear", "MonthNumber"])
)
#quarterly
quarterly_net_sales = (
    merged_data.groupby(["CalendarYear", "CalendarQuarterLabel"])["NetSales"]
    .sum()
    .reset_index()
    .sort_values(["CalendarYear", "CalendarQuarterLabel"])
)

In [20]:
#KPI Total Cost daily
daily_costs = (
    merged_data.groupby("DateKey")["TotalCost"]
    .sum()
    .reset_index()
    .rename(columns={"DateKey": "Date"})
)
#KPI Total Cost monthly
monthly_costs = (
    merged_data.groupby(["CalendarYear", "MonthNumber", "CalendarMonthLabel"])["TotalCost"]
    .sum()
    .reset_index()
    .sort_values(["CalendarYear", "MonthNumber"])
)
#KPI Total Cost quarterly
quarterly_costs = (
    merged_data.groupby(["CalendarYear", "CalendarQuarterLabel"])["TotalCost"]
    .sum()
    .reset_index()
    .sort_values(["CalendarYear", "CalendarQuarterLabel"])
)